# Cutting Stock 

Cada rollo de paple tiene un ancho de 100

Ancho / Demanda:

- 14 / 211
- 31 / 395
- 36 / 610
- 45 / 97

¿Cómo cortar cada rollo de paper para que el número de rollos originales usados sea el menor posible?


### Veamos como formularlo en Jump
**OJO: En este módulo usaremos modelos enteros (que se resuelven con ``Cbc``) y continuos (que se resuelven con ``Clp``)**

In [1]:
using JuMP
using Clp 
using Cbc


Datos del problema (de juguete. Despues probaremos con los correctos):

In [2]:
#anchos
w=[14 31 36 45]
requerimientos=[2; 4; 6; 1]


4-element Array{Int64,1}:
 2
 4
 6
 1

## Ejercicio: Resolver el modelo con la formulación tradicional (usar Cbc para resolverlo)
Recuerdo: Para definir una variable entera o binaria usamos:
``` @variable(model, x, Int) ``` o ``` @variable(model, x, Bin) ```

$$
\begin{align}
\min \sum_{k\in \mathcal{K}} y_k& \\ 
\sum_{k\in \mathcal{K}} x_{ik} &\geq b_i \quad \forall i\in I\\
\sum_{i\in I} w_i x_{ik} &\leq W y_k \quad \forall k\in \mathcal{K}\\
x_{ik}\in\mathbb{N}&, y_k\in \{0,1\}$$

In [3]:
K=13
model = Model(solver=CbcSolver())
@variable(model, x[1:length(requerimientos),1:K] >= 0, Int)
@variable(model, y[1:K], Bin)
@objective(model, Min, sum(y[k] for k=1:K))
for i=1:length(requerimientos)
    @constraint(model, sum(x[i,k] for k=1:K) >= requerimientos[i])
end
for k=1:K
    @constraint(model, sum(w[i]*x[i,k] for i=1:length(requerimientos)) <= 100 * y[k])
end
model




Minimization problem with:
 * 17 linear constraints
 * 65 variables: 13 binary, 52 integer
Solver is CbcMathProg

In [4]:
solve(model)
@show getobjectivevalue(model)
@show getvalue(x)

getobjectivevalue(model) = 5.0
getvalue(x) = 

4×13 Array{Float64,2}:
 2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  0.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  2.0  2.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[2.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 2.0 0.0 0.0 0.0 2.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 2.0 2.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


Inicializamos el ***"problema master"*** con solo 2 variables, de forma de tener una solución inicial:
- ancho (14,31,36,45), cantidad (1,1,0,1), como $x_1$
- ancho (14,31,36,45), cantidad (0,0,2,0), como $x_2$

**\[Recuerdo 1\]** ¿Qué significa cada variable? _El número de rollos de papel que cortaremos con este patrón._

**\[Recuerdo 2\]** Como se ve el problema master?

$$
\begin{align}
\nonumber\min\qquad\qquad\quad &x_1+x_2 \\
s.t.\qquad\left( \begin{array}{c}
1 \\
1 \\
0 \\
1 \end{array} \right)&x_1+\left( \begin{array}{c}
0 \\
0 \\
2 \\
0 \end{array} \right)x_2 \ge \left( \begin{array}{c}
211 \\
395 \\
610 \\
97 \end{array} \right) \\ 
&x_1,x_2\ge0
\end{align}$$

In [2]:
#anchos
w=[14 31 36 45]
requerimientos=[211; 395; 610; 97]
patrones = [[1,1,0,1],[0,0,2,0]]

2-element Array{Array{Int64,1},1}:
 [1, 1, 0, 1]
 [0, 0, 2, 0]

In [6]:
master = Model(solver=ClpSolver())  

#una variable por cada patrón
@variable(master, x[1:length(patrones)] >= 0)

#definimos una referencia a las restricciones para recuperar los duales.
@defConstrRef myCons[1:4]
for i=1:4
    myCons[i] = @constraint(master, sum(x[p]*patrones[p][i] for p in 1:length(patrones))>=requerimientos[i])
end

#Objetivo
@objective(master, Min, sum(x))
master

Minimization problem with:
 * 4 linear constraints
 * 2 variables
Solver is ClpMathProg

In [7]:
status=solve(master)
@show getValue(x)

#obtenemos el optimo
println("\nSolución óptima:\n")

println("anchos: ", w)

epsilon=1e-6

for i=1:length(patrones)
   
    if getValue(x[i])>epsilon 
        println("Patron actual: ", patrones[i], ", Número de rollos con este patrón: ", getValue(x[i]))
    end
end

getValue(x) = [395.0, 305.0]

Solución óptima:

anchos: [14 31 36 45]
Patron actual: 

Necesitamos con ese patrón 700 rollos.  ¿Óptimo?  Claramente no. 

Generemos entonces nuevos patrones basados en los costos reducidos. Denotemos $r=(r_1,r_2,r_3,r_4)$ como los optimos duales de las restricciones 1, 2, 3, 4. El costo reducido de una variable $x_k$, con patrón $A_k$ puede calcularse como
$$rc(x_k)=1-A_k^Tr$$

Queremos buscar una variable $x_k$ tal que $rc(x_k)<0$, lo que podemos hacer resolviendo el siguiente problema de la mochila:

$$\begin{align}
z^*=\max\qquad &r_1a_{k,1}+r_2a_{k,2}+r_3a_{k,3}+r_4a_{k,4} \\
s.t.\qquad &14a_{k,1}+31a_{k,2}+36a_{k,3}+45a_{k,4}\le 100 \\
&a_{k,1},a_{k,2},a_{k,3},a_{k,4}\ge0,~\textrm{y enteros}
\end{align}$$

Si $z^*>1$, entonces $x_k$ con un patrón de corte $(a_{k,1},a_{k,2},a_{k,3},a_{k,4})$ debe ser agregado al _master_ y re-resolver el problema.

*OJO:* como este modelo es entero, debemos usar Cbc como solver esta vez.

In [8]:
using Cbc
duales=getDual(myCons)[1:4]
sub = Model(solver=CbcSolver())  

#variable que define el patrón. Esta es entera.
@variable(sub, a[1:4]>=0, Int)

#restricción para que el patrón sea factible
@constraint(sub, sum(a[i]*w[i] for i in 1:4)<=100)

#define objetivo
@objective(sub, Max, sum(duales[i]*a[i] for i in 1:4))

sub



Maximization problem with:
 * 1 linear constraint
 * 4 variables: 4 integer
Solver is CbcMathProg

[1, 1, 0, 1], Número de rollos con este patrón: 395.0
Patron actual: [0, 0, 2, 0], Número de rollos con este patrón: 305.0


In [9]:
status=solve(sub)


nuevoPatron=getValue(a)[1:4]
println("Costo reducido: ", 1-getobjectivevalue(sub))

println("Anchos: ", w)

nuevoPatron = round(nuevoPatron)

println("\nNuevo patrón de corte: ", nuevoPatron)

push!(patrones, nuevoPatron)


Costo reducido: -2.0
Anchos: [14 31 36 45]


3-element Array{Array{Int64,1},1}:
 [1, 1, 0, 1]
 [0, 0, 2, 0]
 [0, 3, 0, 0]

El costo reducido es $(1-3)=-2<0$. Agregamos esta variable entonces al ***"master problem"***.

In [10]:
master

Minimization problem with:
 * 4 linear constraints
 * 2 variables
Solver is ClpMathProg

JuMP (y casi todos los solvers de optimización) permiten definir un modelo tanto por filas (es decir, restricción por restricción), como por columa (es decir, variable por variable). De esta forma, podemos agregar facilmente una nueva variable en este caso, simplemente necesitamos decir:

- el objetivo de la variable
- en que restricciones aparece la variable, y con que coeficientes

In [11]:
@variable(master, z>=0, objective=1, inconstraints=myCons, coefficients=nuevoPatron)
master

Minimization problem with:
 * 4 linear constraints
 * 3 variables
Solver is ClpMathProg

In [12]:
master

Minimization problem with:
 * 4 linear constraints
 * 3 variables
Solver is ClpMathProg


Nuevo patrón de corte: [0.0, 3.0, 0.0, 0.0]


In [13]:
#Resolvermos
status=solve(master)

#get the optimal solution
println("\nSolución óptima:\n")

println("Anchos: ", w)

for i=1:length(x)
   
    if getValue(x[i])>epsilon
        println("Patron inicial: ", patrones[i], ", Número de rollos con este patrón: ", getValue(x[i]))


    end
end


if getValue(z)>epsilon
    println("Patron nuevo: ", nuevoPatron, ", Número de rollos con este patrón: ", getValue(z))
end


Como vemos, el objetivo se reduce a 577.3 gracias a la nueva variable. 

### Pongamos ahora todo junto! (Sugerencia: reiniciar el Kernel)

In [1]:
using JuMP  
using Clp
using Cbc
master = Model(solver=ClpSolver(LogLevel=0)) 

w=[14 31 36 45]
requerimientos=[211; 395; 610; 97]
patrones = [[1,1,0,1],[0,0,2,0]]

@variable(master, x[1:length(patrones)] >= 0)
@defConstrRef myCons[1:4]
for i=1:4
    myCons[i] = @constraint(master, sum(x[p]*patrones[p][i] for p in 1:length(patrones))>=requerimientos[i])
end
@objective(master, Min, sum(x))
solve(master)
println("Iteración 1, Objetivo del Master:", getObjectiveValue(master))

# suproblema

duales=getDual(myCons)[1:4]
sub=Model(solver=CbcSolver(logLevel=0))  
w=[14,31,36,45]
@variable(sub, a[1:4]>=0, Int)
@constraint(sub, sum(a[i]*w[i] for i in 1:4)<=100)
@objective(sub, Max, sum(duales[i]*a[i] for i in 1:4))


#Resolvemos el subproblema
solve(sub)
sub_obj=getObjectiveValue(sub);
epsilon=1e-6; 

#guardemos las variables en un arreglo
variables=Variable[]
push!(variables,x[1])
push!(variables,x[2])

iter=2

while sub_obj>1+epsilon  #Mientras costo reducido sea >1

    #Obtenemos el nuevo patrón
    nuevoPatron=getValue(a)[1:4]
    nuevoPatron = round(nuevoPatron)

    #Agregamos la nueva variable z
    @variable(master, z>=0, objective=1, inconstraints=myCons, coefficients=nuevoPatron)
    
    println("\tAgregada una nueva variable con patrón: ", nuevoPatron, ", costo reducido: ", (1-sub_obj))
    
    #agregamos la variable a la lista de variables
    push!(variables, z)
    #agregamos el nuevo patron a la lista de patrones
    push!(patrones, nuevoPatron)
    
    solve(master)
    
    println("\Iteración ",iter, ", Objetivo del Master:", getObjectiveValue(master))
    
    #duales
    duales=getDual(myCons)[1:4]
    
    #modificamos el objetivo del problema sub
    @objective(sub, Max, sum(duales[i]*a[i] for i in 1:4))
    
    solve(sub);
    
    sub_obj=getObjectiveValue(sub)
    
    iter=iter+1
    
end

#imprimo la solución óptima
println("\nSolución Óptima:\n")

println("anchos: ", w)

suma = 0
for i=1:length(variables)
   
    #if getValue(variables[i])>epsilon
        println("Patron: ", patrones[i], ", Número de rollos con este patrón: ", getValue(variables[i]))
        suma=suma+getValue(variables[i])
    #end
end

println("Número total de rollos requeridos: ", suma)

Iteración 1, Objetivo del Master:700.0
	Agregada una nueva variable con patrón: [0.0, 3.0, 0.0, 0.0], costo reducido: -2.0
Iteración 2, Objetivo del Master:577.3333333333334


	Agregada una nueva variable con patrón: [7.0, 0.0, 0.0, 0.0], costo reducido: -3.666666666666667
Iteración 3, Objetivo del Master:517.6190476190476
Presolve 0 (-1) rows, 0 (-4) columns and 0 (-4) elements
Optimal - objective value 1
After Postsolve, objective 1, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1 - 0 iterations time 0.002, Presolve 0.00
Cbc0045I Solution with objective value -1 saved
	Agregada una nueva variable con patrón: [2.0, 0.0, 2.0, 0.0], costo reducido: -0.2857142857142856
Iteración 4, Objetivo del Master:501.3333333333333
Presolve 0 (-1) rows, 0 (-4) columns and 0 (-4) elements
Optimal - objective value 1
After Postsolve, objective 1, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1 - 0 iterations time 0.002, Presolve 0.00
Cbc0045I Solution with objective value -1 saved
	Agregada una nueva variable con patrón: [0.0, 0.0, 0.0, 2.0], costo reducido: -0.3333333333333335
Iteración 5, Objetivo del Master:485.16666666666663
Presolve 0 (-1) 

Excelente! Necesitamos entonces solo 3 patrones de cortes (y que son muy distintos a los que inicialmente mencionamos)

### Como obtener una solución ENTERA? ###

Generación de columna solo resuelve problemas lineales LP. Sin embargo, el problema es entero (no podemos usar 206.25 rollos  con el primer patrón). ¿Se puede hacer una generación de columnas _entera_? 

*Si*, pero eso requiere una técnica avanzada llamada  [branch-and-price](http://en.wikipedia.org/wiki/Branch_and_price) que es básicamente un árbol de branch-and-bound donde en cada nodo resuelvo un problema de generación de columnas. Desafortunadamente, los solvers comerciales (Gurobi, CPLEX) no resuelven este tipo de problemas. Hasta ahora, el único solver académico que resuelve branch-and-price es [SCIP](http://scip.zib.de/).

En vez de resolver el problema entero a optimalidad, podemos usar dos métodos para generar una solución entera.


#### Metodo 1: Redondeo ####

Una heuristica común es redondear la solución fraccionaria al entero mas _cercano_ y _factible_. Como redondear dependerá del problema. En este problema, debemos redondear _hacia arriba_ para poder estar seguros de que la solución es factible. 

In [15]:
println("\nSolución entera basada en redondeo:\n")

println("Ancho: ", w)

suma=0.0

for i=1:length(variables)
    if getValue(variables[i])>epsilon
        println("Patron: ", patrones[i], ", Número de rollos con este patrón: ", ceil(getValue(variables[i])))
        suma=suma+ceil(getValue(variables[i]))
    end
end


println("Número total de rollos requeridos: ", suma)

Ok, tenemos una solución entera que usa 454 rollos en total. Recordemos que la solución LP nos da valor 452.25. O sea, aunque sea una heurística, sabemos que estamos a menos de un 0.4% del óptimo.

#### Método 2: Branch-and-Bound con las columnas generadas ####

Otra práctica es usar directamente las columnas generadas pero forzando que sean enteras. Esto no entrega un óptimo, ya que otra columna podría agregarse, pero puede entregar una  solución suficientemente buena.



OJO: Por ser ahora un problema entero, debemos re-formularlo con Cbc como solver

In [2]:
patrones

7-element Array{Array{Int64,1},1}:
 [1, 1, 0, 1]
 [0, 0, 2, 0]
 [0, 3, 0, 0]
 [7, 0, 0, 0]
 [2, 0, 2, 0]
 [0, 0, 0, 2]
 [0, 2, 1, 0]

In [3]:
master = Model(solver=CbcSolver(logLevel=0))  

#una variable por cada patrón
@variable(master, x[1:length(patrones)] >= 0, Int)

#definimos una referencia a las restricciones para recuperar los duales.
@defConstrRef myCons[1:4]
for i=1:4
    myCons[i] = @constraint(master, sum(x[p]*patrones[p][i] for p in 1:length(patrones))>=requerimientos[i])
end

#Objetivo
@objective(master, Min, sum(x))

solve(master)




println("\nSolución entera usando las columnas generadas:\n")

println("anchos: ", w)

suma=0

for i=1:length(x)
   
    if getValue(x[i])>epsilon
        println("Patron: ", patrones[i], ", Número de rollos con este patrón: ", ceil(getValue(x[i])))
        suma=suma+getValue(x[i])
    end
end


println("Número total de rollos requeridos: ", suma)


Solución entera usando las columnas generadas:

anchos: [14, 31, 36, 45]
Patron: [1, 1, 0, 1], Número de rollos con este patrón: 1.0
Patron: [0, 0, 2, 0], Número de rollos con este patrón: 102.0
Patron: [2, 0, 2, 0], Número de rollos con este patrón: 106.0
Patron: [0, 0, 0, 2], Número de rollos con este patrón: 48.0
Patron: [0, 2, 1, 0], Número de rollos con este patrón: 197.0
Número total de rollos requeridos: 453.0


Efectivamente, podiamos ahorrarnos 1 rollo extra usando el método 2.